In [6]:
import os

os.environ["SEED"] = "0"
os.environ["TRAIN_DIR"] = "TRAIN"

import numpy as np

from src.aac.AAC import AAC
from src.constant import DATA_DIR, DATABASE_DIR, MAIN_DIR
from src.database import DB
from src.database.queries import get_model_training_data, get_solvers_count
from src.instance.TSP_Instance import TSP_from_index_file
from src.solver.Portfolio import Portfolio
from src.solver.TSP_LKH_Solver import TSP_LKH_Solver
from src.log import logger
from src.surrogate.SurrogatePolicy import SurrogatePolicy, TestSurrogatePolicy
from src.surrogate.wrapper import EmptyWrapper, SurvivalFunctionWrapper

In [7]:
import pickle
with open(MAIN_DIR / "archive" / "phase1" / "results" / "permutation" / "HO" / "coxph_incumbent.pkl", "rb") as f:
    coxph_incumbent = pickle.load(f)

In [8]:
train_instances = TSP_from_index_file(
    filepath=DATA_DIR / "TSP" / "TRAIN" / "index.json",
    cut_off_cost=100,
    cut_off_time=10,
    n=10,
)
test_instances = TSP_from_index_file(
    filepath=DATA_DIR / "TSP" / "TEST" / "index.json",
    cut_off_cost=1000,
    cut_off_time=100,
    n=250,
)

In [9]:
for instance in train_instances:
    instance.cut_off_time = round(10 * ((instance.n_cities / 600) ** 2.2), 2)
    instance.cut_off_cost = 10 * instance.cut_off_time

In [10]:
# surrogate_policy = SurrogatePolicy(
#     estimator_wrapper=EmptyWrapper(),
#     first_fit_solver_count=1,
#     refit_solver_count=2,
# )

# surrogate_policy = TestSurrogatePolicy(
#     estimator_wrapper=EmptyWrapper(),
#     first_fit_solver_count=1,
#     refit_solver_count=2,
# )

surrogate_policy = TestSurrogatePolicy(
    estimator_wrapper=SurvivalFunctionWrapper(**coxph_incumbent),
    first_fit_solver_count=5,
    refit_solver_count=2,
)

In [11]:
SOLVERS_N = 2
ATTEMPTS_N = 3
MAX_ITER = 10

solvers = []
largest_marginal_contribution_solver = None

for solver_i in range(SOLVERS_N):
    logger.info(f"Solver {solver_i + 1}/{SOLVERS_N}")

    best_cost = np.inf
    best_solver = None
    attempt_solvers = []

    for attempt_i in range(ATTEMPTS_N):
        logger.info(f"Attempt {attempt_i + 1}/{ATTEMPTS_N}")

        if largest_marginal_contribution_solver is not None:
            new_solver = largest_marginal_contribution_solver.copy()
        else:
            new_solver = TSP_LKH_Solver()

        iteration_solvers = solvers + [new_solver]

        portfolio = Portfolio.from_iterable(iteration_solvers)
        aac = AAC(
            portfolio=portfolio,
            instance_list=train_instances,
            prefix=f"config;solver={solver_i+1};attempt={attempt_i+1}",
            max_iter=MAX_ITER,
            i=solver_i,
            calculate_features=False,
            surrogate_policy=surrogate_policy,
        )
        portfolio = aac.configure()
        result = portfolio.evaluate(  # fix cut-off times before validation
            instance_list=train_instances,
            prefix=f"validate;solver={solver_i+1};attempt={attempt_i+1}",
            cache=True,
        )
        attempt_solvers.append(portfolio[solver_i])
        logger.info(
            f"Attempt {attempt_i + 1}/{ATTEMPTS_N}: cost = {result.cost:.2f}"
        )
        if result.cost < best_cost:
            best_cost = result.cost
            best_solver = portfolio[solver_i]

    solvers.append(best_solver)
    logger.info(f"Solver {solver_i + 1}/{SOLVERS_N}: best cost = {best_cost:.2f}")

    if solver_i < SOLVERS_N - 1:
        largest_marginal_contribution_solver = None
        best_cost = np.inf
        for attempt_i, solver in enumerate(attempt_solvers):
            if solver != best_solver:
                portfolio = Portfolio.from_iterable(solvers + [solver])
                result = portfolio.evaluate(
                    instance_list=train_instances,
                    prefix=f"largest_marginal_contribution;attempt={attempt_i+1}",
                    cache=True,
                )
                if result.cost < best_cost:
                    best_cost = result.cost
                    largest_marginal_contribution_solver = solver

[2025-04-29 13:59:26] INFO      Solver 1/2
[2025-04-29 13:59:26] INFO      Attempt 1/3
[2025-04-29 13:59:26] DEBUG     AAC(prefix=config;solver=1;attempt=1, iter=1/10)
[2025-04-29 13:59:26] DEBUG     SurrogatePolicy(estimator_wrapper=<src.surrogate.wrapper.SurvivalFunctionWrapper.SurvivalFunctionWrapper object at 0x000001D725F2C550>, first_fit_solver_count=5, refit_solver_count=2, last_fit_solver_count=0, is_fitted=False)
[2025-04-29 13:59:26] DEBUG     SurrogatePolicy.notify_iter(iter=1, solver_count=0)
[2025-04-29 13:59:26] DEBUG     Portfolio.evaluate(config;solver=1;attempt=1;aac_iter=1)
[2025-04-29 13:59:26] DEBUG     Portfolio(size=1)[Solver(id=52980316141715150)]
[2025-04-29 13:59:26] DEBUG     solve(prefix=config;solver=1;attempt=1;aac_iter=1, solver=Solver(id=52980316141715150), instance=TSP_Instance(filepath=TSP/TRAIN/cluster_netgen/000.tsp))
[2025-04-29 13:59:26] DEBUG     solve(prefix=config;solver=1;attempt=1;aac_iter=1, solver=Solver(id=52980316141715150), instance=TSP_In

In [ ]:
solver = portfolio[0]
instance = train_instances[0]
X = np.concatenate([solver.get_array(), instance.get_array()])
X = X.reshape(1, -1)
X.shape

In [ ]:
surrogate_policy.estimator_wrapper.predict(X, np.array([instance.cut_off_time]))

In [ ]:

cost = estimator_wrapper.predict(X, instance.cut_off_time)[0]

In [ ]:
surrogate_policy.estimator_wrapper

In [ ]:
DB().get_solvers()

In [ ]:
db = DB(db_path=DATABASE_DIR / "test-2025_04_29_06_36_27.db")

In [ ]:
X, y, cut_off = get_model_training_data(db)

In [ ]:
cut_off.shape

In [ ]:
get_solvers_count(db)

In [ ]:
db.query2df(f"select count(*) from {DB.SCHEMA.SOLVERS}").iloc[0, 0]

In [ ]:
query = f"""
select 
    {db.SCHEMA.RESULTS}.cost,
    {db.SCHEMA.RESULTS}.cut_off_time,
    {db.SCHEMA.SOLVERS}.*,
    {db.SCHEMA.INSTANCES}.*
from {db.SCHEMA.RESULTS}
join {db.SCHEMA.INSTANCES} on {db.SCHEMA.RESULTS}.instance_id = {db.SCHEMA.INSTANCES}.id
join {db.SCHEMA.SOLVERS} on {db.SCHEMA.RESULTS}.solver_id = {db.SCHEMA.SOLVERS}.id
where {db.SCHEMA.RESULTS}.cached = 0 and {db.SCHEMA.RESULTS}.surrogate = 0
"""
df = db.query2df(query)

In [ ]:
df = df.drop(columns=["id", "filepath", "optimum"])
df = df.dropna()


In [ ]:
df

In [ ]:
y = df["cost"].to_numpy()
cut_off = df["cut_off_time"].to_numpy()
y = np.where(y >= cut_off, cut_off, y)
X = df.drop(columns="cost").to_numpy()

In [ ]:
y

In [ ]:
cut_off

In [ ]:
X.shape